In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
class PreProcessing():
    def __init__(self, df : pd.DataFrame = pd.DataFrame) :
        self.df = df
        
    
    def Information (self , df) :
        self.df = df
        df.replace('?' , np.nan , inplace = True )
        nulls = (df.shape[0] - df.dropna(axis = 0).shape[0])
        nulls_per = (df.shape[0] - df.dropna(axis = 0).shape[0]) / df.shape[0]
        dup , dup_per = df.duplicated().sum() , (df.duplicated().sum()) / df.shape[0]
        numerical_columns = list((df.select_dtypes(include=np.number)).columns)
        categorical_columns = list(df.select_dtypes(include = ['object']).columns)
        numerical_col_dict = {}
        for col in numerical_columns :
            if df[col].nunique() > 10 :
                numerical_col_dict[col] = 'Continuous' 

            else :
                numerical_col_dict[col] = 'Discrete'
        categorical_col_dict = {}
        for col in categorical_columns :
            if df[col].nunique() > 10 :
                categorical_col_dict[col] = 'Continuous' 

            else :
                categorical_col_dict[col] = 'Classes'
        x = []
        for col in df.columns : 
            x.append([col , df[col].unique() ,df[col].nunique(),df[col].dtype,df[col].isna().sum(),int((df[col].isna().sum())/df.shape[0] * 100)  ])

        x = pd.DataFrame(x ,columns=['Feature','Unique_Values','No.OfUniqueValues','DataType','NoOfNulls','Nulls %'])
        
        print('\n','There is : {}  Nulls, Representing {} % of The Data'.format(nulls , '%.2f'%nulls_per ))
        print('\n','There is : {}  Duplicated, Representing {} % of The Data'.format(dup , '%.2f'%dup_per))
        print('\n','Numerical Columns : {}'.format(numerical_col_dict))
        print('\n','Categorical Columns : {}'.format(categorical_col_dict))
        
        return x
    
    def ImputeNulls(self , df , col_method : dict = {} ,col_value = {} ,type_method : dict = {} ) :
        self.df = df
        self.col_method = col_method
        self.type_method = type_method
        self.col_value = col_value
        df_copy = df.copy(deep = True)
        if len(col_method) == 0 & len(type_method) == 0  : 
            numerical_columns = list((df_copy.select_dtypes(include=np.number)).columns)
            categorical_columns = list(df_copy.select_dtypes(include = ['object']).columns)

            for col in numerical_columns :
                df_copy[col] = df_copy[col].interpolate()
                
            df_copy[categorical_columns] = df_copy[categorical_columns].apply(lambda x:x.fillna(x.value_counts().index[0]))
    
                
        if len(col_method) != 0 :
            for col in col_method.keys() :
                df_copy[col] = df_copy[col].fillna(method = col_method[col])
        
        if len(col_value) != 0 :
            for col in col_value.keys() :
                df_copy[col] = df_copy[col].fillna(value = col_value[col])
        
        if len(type_method) != 0 :
            numerical_columns = list((df_copy.select_dtypes(include=np.number)).columns)
            categorical_columns = list(df_copy.select_dtypes(include = ['object']).columns)
            for col in numerical_columns :
                df_copy[col] = df_copy[col].fillna(method = type_method['int'])
                
            for col in categorical_columns :
                df_copy[col] = df_copy[col].fillna(method = type_method['object'])
                
        return df_copy
    
    
    def Oultiers(self , df , display = False , drop = False , drop_order = 1 ,columns_to_display = [], columns_to_drop = []) :
        self.df = df
        self.display = display
        self.drop = drop
        self.drop_order = drop_order
        self.columns_to_display = columns_to_display
        self.columns_to_drop = columns_to_drop
        df_copy = df.copy(deep = True)
        numerical_columns = list((df_copy.select_dtypes(include=np.number)).columns)


        if display == True :
            if (len(columns_to_display) != 0) :
                numerical_columns = columns_to_display
            from math import ceil
            x = ceil(len(numerical_columns)/2)
            if len(numerical_columns) >= 24 :
                fig = plt.figure(figsize=(128,96))
            else : 
                fig = plt.figure(figsize=(24, 12))

            sns.set_theme(style="whitegrid", palette="pastel")
            plt.subplots_adjust(hspace=0.5)
            subfigs = fig.subfigures(len(numerical_columns), 1)
            for idx , i in enumerate(numerical_columns) :
                (ax1, ax2) = subfigs[idx].subplots(1, 2)
                sns.boxplot(x = df_copy[i] ,data = df_copy , ax = ax1 )
                sns.histplot(x = df_copy[i], kde=True , ax = ax2 )
                subfigs[idx].subplots_adjust(bottom=0.5, right=0.8, top=0.75)
                ax1.set_title(i)
                ax2.set_title(i)

        if drop == True :
            if (len(columns_to_drop) != 0) :
                numerical_columns = columns_to_drop
            elif drop_order < 1 :
                drop_order = 1

            import scipy.stats as stats

            while drop_order != 0 :
                for col in  numerical_columns :
                    p_value = stats.normaltest(df_copy[col])
                    if p_value[1] >= .05 :                                      ## col is normally distributed
                        filt = (df_copy[col]-df_copy[col].mean())/df_copy[col].std()
                        indexes = (filt[filt > 3]).index
                        df_copy.drop(indexes , inplace = True )
                        df_copy.reset_index(drop=True , inplace=True)

                    else :                                                     ## col is Skewed 

                        q1 = df_copy[col].quantile(0.25)
                        q3 = df_copy[col].quantile(0.75)
                        iqr = q3 - q1
                        lower = q1 - 1.5*iqr
                        if lower < 0 :
                            lower = 0
                        higher = q3 + 1.5*iqr
                        df_copy = df_copy [df_copy[col] >= lower] 
                        df_copy = df_copy [df_copy[col] <= higher ]
                        df_copy.reset_index(drop=True , inplace=True)

                drop_order = drop_order - 1
                
        return df_copy

       
        
    def Encode_Categorical(self , df , col_method={} ,one_hot = False , labeling = False ,defult = False ) :
        self.df = df 
        self.one_hot = one_hot
        self.labeling = labeling
        self.col_method = col_method
        df_copy = df.copy(deep = True)
        from sklearn import preprocessing
        
        if len(col_method) != 0 :
            label_cols = []
            one_hot_cols = []
            
            for col in col_method.keys() :
                if 'label' in col_method[col] :
                    label_cols.append(col)
                else :
                    one_hot_cols.append(col)
            if len(label_cols) !=0 :
                label_encoder = preprocessing.LabelEncoder()
                for col in label_cols :
                    df_copy[col]= label_encoder.fit_transform(df_copy[col])
            if len(one_hot_cols) !=0 :
                dummies = pd.get_dummies(one_hot_cols , drop_first=True)
                df_copy.drop(one_hot_cols,axis=1,inplace=True)
                df_copy = df_copy.join(dummies)

        
        elif defult == True :
            categorical_columns = list(df_copy.select_dtypes(include = ['object']).columns)
            one_hot_encoding = []
            label_encoder = preprocessing.LabelEncoder()
            for col in categorical_columns :
                if df_copy[col].nunique() < 20 :
                    one_hot_encoding.append(col)
                    
                else :
                    df_copy[col]= label_encoder.fit_transform(df_copy[col])
                    
            df_copy = pd.get_dummies(df_copy , drop_first=True)
                    
                
                    
        
        elif one_hot == True :
            df_copy = pd.get_dummies(df_copy , drop_first=True)
            
        elif labeling == True :
            from sklearn import preprocessing
            label_encoder = preprocessing.LabelEncoder()
            for col in df_copy.columns :
                df_copy[col]= label_encoder.fit_transform(df_copy[col])
                
                
        return df_copy
    
    
    def Model (self , df , model ,target_name , problem ,test_size = 0.3 , random_state = 42 ) :
        self.df = df
        self.model = model
        self.target_name = target_name
        self.problem = problem
        self.test_size = test_size
        self.random_state = random_state
        
        
        from sklearn.model_selection import train_test_split
        from sklearn.metrics import accuracy_score ,recall_score,precision_score,f1_score ,classification_report , r2_score
        x = df.drop([target_name] , axis = 1)
        y = df[target_name]
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size, random_state = random_state)
        
        list_of_models = [model]
        if problem == 'Classification' :
            classification_report = pd.DataFrame(columns=['Accuracy','F1_score','Precision','Recall'])
            for model in list_of_models :
                model = model
                model.fit(x_train , y_train)
                for i in range(2) :

                    if i == 0 :
                        to_pred = x_train
                        pred = y_train
                        title = 'Train'

                    else :
                        to_pred = x_test
                        pred = y_test
                        title = 'Test'
                    y_pred = model.predict(to_pred)
                    acc = round(accuracy_score(pred , y_pred)*100)
                    f1 = round(f1_score(pred , y_pred)*100)
                    prec = round(precision_score(pred , y_pred)*100)
                    recall = round(recall_score(pred , y_pred)*100)
                    d = pd.DataFrame(data=np.array([acc,f1,prec,recall]).reshape(1,4) 
                                 , columns=['Accuracy' , 'F1_score' , 'Precision' , 'Recall'])  
                    classification_report = pd.concat([classification_report , d])
                    classification_report.rename( index= { 0 :'{} _ {} Details'.format(model , title) } ,inplace=True )
                    
        elif problem == 'Regression' :
            classification_report = pd.DataFrame(columns=['Score'])
            for model in list_of_models :
                model = model
                model.fit(x_train , y_train)
                for i in range(2) :

                    if i == 0 :
                        to_pred = x_train
                        pred = y_train
                        title = 'Train'

                    else :
                        to_pred = x_test
                        pred = y_test
                        title = 'Test'
                    y_pred = model.predict(to_pred)
                    acc = round(r2_score(pred , y_pred)*100)
                    d = pd.DataFrame(data=np.array([acc]).reshape(1,1) 
                                 , columns=['Score'])  
                    classification_report = pd.concat([classification_report , d])
                    classification_report.rename( index= { 0 :'{} _ {} Details'.format(model , title) } ,inplace=True )
                    
        print(classification_report)
        return model


In [4]:
pre=PreProcessing()

In [7]:
data=pd.read_csv('application_record.csv')
data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [8]:
data2=pd.merge(left= data, right=credit, how='inner',on= ['ID'])
pre.Information(data2)

NameError: name 'credit' is not defined

In [5]:
data2.head()

NameError: name 'data2' is not defined

In [9]:
credit=pd.read_csv('credit_record.csv')
credit.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [11]:
data2=pd.merge(left= data, right=credit, how='inner',on= ['ID'])
pre.Information(data2)


 There is : 240048  Nulls, Representing 0.31 % of The Data

 There is : 0  Duplicated, Representing 0.00 % of The Data

 Numerical Columns : {'ID': 'Continuous', 'CNT_CHILDREN': 'Discrete', 'AMT_INCOME_TOTAL': 'Continuous', 'DAYS_BIRTH': 'Continuous', 'DAYS_EMPLOYED': 'Continuous', 'FLAG_MOBIL': 'Discrete', 'FLAG_WORK_PHONE': 'Discrete', 'FLAG_PHONE': 'Discrete', 'FLAG_EMAIL': 'Discrete', 'CNT_FAM_MEMBERS': 'Discrete', 'MONTHS_BALANCE': 'Continuous'}

 Categorical Columns : {'CODE_GENDER': 'Classes', 'FLAG_OWN_CAR': 'Classes', 'FLAG_OWN_REALTY': 'Classes', 'NAME_INCOME_TYPE': 'Classes', 'NAME_EDUCATION_TYPE': 'Classes', 'NAME_FAMILY_STATUS': 'Classes', 'NAME_HOUSING_TYPE': 'Classes', 'OCCUPATION_TYPE': 'Continuous', 'STATUS': 'Classes'}


,Feature,Unique_Values,No.OfUniqueValues,DataType,NoOfNulls,Nulls %
0,ID,"[5008804, 5008805, 5008806, 5008808, 5008809, ...",36457,int64,0,0
1,CODE_GENDER,"[M, F]",2,object,0,0
2,FLAG_OWN_CAR,"[Y, N]",2,object,0,0
3,FLAG_OWN_REALTY,"[Y, N]",2,object,0,0
4,CNT_CHILDREN,"[0, 1, 3, 2, 4, 5, 14, 19, 7]",9,int64,0,0
5,AMT_INCOME_TOTAL,"[427500.0, 112500.0, 270000.0, 283500.0, 13500...",265,float64,0,0
6,NAME_INCOME_TYPE,"[Working, Commercial associate, Pensioner, Sta...",5,object,0,0
7,NAME_EDUCATION_TYPE,"[Higher education, Secondary / secondary speci...",5,object,0,0
8,NAME_FAMILY_STATUS,"[Civil marriage, Married, Single / not married...",5,object,0,0
9,NAME_HOUSING_TYPE,"[Rented apartment, House / apartment, Municipa...",6,object,0,0


In [13]:
data3=pd.concat([data,credit],axis=1)
data3.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ID,MONTHS_BALANCE,STATUS
0,5008804.0,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,5001711,0,X
1,5008805.0,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,5001711,-1,0
2,5008806.0,M,Y,Y,0.0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,-1134.0,1.0,0.0,0.0,0.0,Security staff,2.0,5001711,-2,0
3,5008808.0,F,N,Y,0.0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,5001711,-3,0
4,5008809.0,F,N,Y,0.0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,5001712,0,C


In [ ]:
pre.Information(data2)

In [14]:
data2.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0,C
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-1,C
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-2,C
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-3,C
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-4,C


In [15]:
data2['NAME_HOUSING_TYPE'].unique()
#property, marital status & its relation to credit score, income & its relation to 

array(['Rented apartment', 'House / apartment', 'Municipal apartment',
       'With parents', 'Co-op apartment', 'Office apartment'],
      dtype=object)

In [17]:
#df3=pd.merge(data1,data2,left_index=True,right_index=True)
pre.Information(df3)

NameError: name 'df3' is not defined

In [ ]:

def ourInfo(data):
    df=data.copy(deep=True)
    x = []
    for col in df.columns : 
        x.append([col , df[col].unique() ,df[col].nunique(),df[col].dtype,df[col].isna().sum() ])

    info=pd.DataFrame(x ,columns=['Feature','Unique_Values','No.OfUniqueValues','DataType','NoOfNulls'])
    from IPython.display import display
    display(info)
    


In [ ]:
pre=PreProcessing()

In [ ]:
df3 = pd.merge(data1, data2, left_index=True, right_index=True)
df3

In [ ]:
pre.Information(df3)

In [ ]:
df3['OCCUPATION_TYPE'].unique()

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
def ourInfo(data):
    df=data.copy(deep=True)
    x = []
    for col in df.columns : 
        x.append([col , df[col].unique() ,df[col].nunique(),df[col].dtype,df[col].isna().sum() ])

    info=pd.DataFrame(x ,columns=['Feature','Unique_Values','No.OfUniqueValues','DataType','NoOfNulls'])
    from IPython.display import display
    display(info)
    


In [ ]:
df3

In [ ]:
df3=pre.ImputeNulls(df3)

In [ ]:
df3=pre.Encode_Categorical(df3,defult=True)

In [ ]:
df3

In [ ]:
data2['AMT_INCOME_TOTAL']


In [ ]:
sns.histplot(x=data2['AMT_INCOME_TOTAL'],hue=data2['CODE_GENDER'])

In [ ]:
sns.boxplot(data=data2['AMT_INCOME_TOTAL'])

In [ ]:
sns.boxplot(data=data2,x=data2['AMT_INCOME_TOTAL']/1000,hue=data2['CODE_GENDER'])

In [ ]:
d = pd.read_csv('https://raw.githubusercontent.com/mGalarnyk/Python_Tutorials/master/Kaggle/BreastCancerWisconsin/data/data.csv')
d.head()

In [ ]:
pre=PreProcessing()
pre.Information(data2)

In [ ]:
data22=pre.Encode_Categorical(df=data2, defult=True)
data22

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
data22.corr()

In [ ]:
data22.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression 

In [ ]:
X=data22.drop(['Class'],axis=1)
y=data22['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [ ]:
lr=LinearRegression()

In [ ]:
data2